The following exercises use 2019 flight statistics from the United States Department of Transportation’s Bureau of Transportation Statistics (available [here](https://www.transtats.bts.gov/DL_SelectFields.asp?gnoyr_VQ=FMF&QO_fu146_anzr=Nv4%20Pn44vr45) and in this repository as `data/T100_MARKET_ALL_CARRIER.zip`). Each row contains information about a specific route for a given carrier in a given month (e.g., JFK &rarr; LAX on Delta Airlines in January). There are 321,409 rows and 41 columns. Note that you don't need to unzip the file to read it in with `pd.read_csv()`.

This continues last week's exercises.

In [1]:
import pandas as pd

df = pd.read_csv("data/T100_MARKET_ALL_CARRIER.zip")

df.columns = df.columns.str.lower()

df

,passengers,freight,mail,distance,unique_carrier,airline_id,unique_carrier_name,unique_carrier_entity,region,carrier,...,dest_state_nm,dest_country,dest_country_name,dest_wac,year,quarter,month,distance_group,class,data_source
0,0.0,53185.0,0.0,8165.0,EK,20392,Emirates,9678A,I,EK,...,Texas,US,United States,74,2019,1,3,17,G,IF
1,0.0,9002.0,0.0,6849.0,EK,20392,Emirates,9678A,I,EK,...,New York,US,United States,22,2019,1,3,14,G,IF
2,0.0,2220750.0,0.0,7247.0,EK,20392,Emirates,9678A,I,EK,...,Illinois,US,United States,41,2019,1,3,15,G,IF
3,0.0,1201490.0,0.0,8165.0,EK,20392,Emirates,9678A,I,EK,...,NaN,AE,United Arab Emirates,678,2019,1,3,17,G,IF
4,0.0,248642.0,0.0,6849.0,EK,20392,Emirates,9678A,I,EK,...,NaN,AE,United Arab Emirates,678,2019,1,3,14,G,IF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321404,96984.0,505308.0,15979.0,1448.0,AS,19930,Alaska Airlines Inc.,06031,D,AS,...,Washington,US,United States,93,2019,3,7,3,F,DU
321405,97011.0,473666.0,55963.0,1448.0,AS,19930,Alaska Airlines Inc.,06031,D,AS,...,Alaska,US,United States,1,2019,2,6,3,F,DU
321406,97098.0,592775.0,18834.0,1448.0,AS,19930,Alaska Airlines Inc.,06031,D,AS,...,Washington,US,United States,93,2019,3,8,3,F,DU
321407,97329.0,210292.0,5431.0,404.0,DL,19790,Delta Air Lines Inc.,01260,D,DL,...,Georgia,US,United States,34,2019,1,3,1,F,DU


#### __Exercise 8__:

Find the top 5 carriers for median route distance (i.e., for all origin-destination pairs that each carrier has, find the median distance after removing duplicates).

In [20]:
# df_unique = df.drop_duplicates(subset=['unique_carrier', 'distance'])
# median_distances = df_unique.groupby('unique_carrier')['distance'].median()
# top_5 = median_distances.sort_values()
# top_5

df_unique = df[["unique_carrier_name", "origin", "distance", "dest"]].drop_duplicates()

df_unique.groupby("unique_carrier_name")["distance"].median().nlargest(5)

unique_carrier_name
Singapore Airlines Ltd.                                       8068.0
Cathay Pacific Airways Ltd.                                   8020.0
Qantas Airways Ltd.                                           7886.0
Longtail Aviation Ltd.                                        7855.5
National Aviation Company of India Limited d/b/a Air India    7798.0
Name: distance, dtype: float64

#### __Exercise 9:__ 
Find the total cargo transported (mail + freight) and mean distance traveled for the 10 carriers that transported the most cargo.

In [29]:
# total_cargo = df[["unique_carrier_name", "mail", "freight", "distance"]].drop_duplicates()
# total_cargo["cargo"] = df["mail"] + df["freight"]
# total_cargo.groupby("unique_carrier_name")["distance"].median().nlargest(10)

total_cargo = df[["unique_carrier_name", "mail", "freight", "distance"]]

total_cargo["cargo"] = df["mail"] + df["freight"]

total_cargo.groupby("unique_carrier_name")[["cargo", "distance"]]\
.agg(dict(cargo="sum", distance="mean")).nlargest(10, "cargo")

/tmp/ipykernel_4607/2555868079.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_cargo["cargo"] = df["mail"] + df["freight"]


,cargo,distance
unique_carrier_name,,
Federal Express Corporation,1.270966e+10,1121.887981
United Parcel Service,9.173867e+09,1030.812650
Atlas Air Inc.,3.356847e+09,1900.987202
United Air Lines Inc.,1.577892e+09,1810.656058
American Airlines Inc.,1.353074e+09,1583.185048
Kalitta Air LLC,1.272180e+09,2519.580343
Polar Air Cargo Airways,1.199386e+09,3111.794118
Delta Air Lines Inc.,1.129524e+09,1612.688752
China Airlines Ltd.,8.370799e+08,5828.277778


#### __Exercise 10__:

Which 10 carriers flew the most passengers out of the United States to another country?

In [45]:
df_outbound = df[(df.origin_country_name == "United States") & (df.dest_country_name != "United States")]

df_outbound.groupby("unique_carrier_name")["passengers"].sum().nlargest(10)

unique_carrier_name
American Airlines Inc.       14867653.0
United Air Lines Inc.        14427923.0
Delta Air Lines Inc.         13054230.0
JetBlue Airways               4522492.0
British Airways Plc           3758945.0
Lufthansa German Airlines     3123611.0
Westjet                       2626600.0
Air Canada                    2540855.0
Southwest Airlines Co.        2146960.0
Virgin Atlantic Airways       2074735.0
Name: passengers, dtype: float64